<style>
pre > code {
    background-color: #3A3960 !important;
    padding: 10px;
    display: block;
    border-radius: 5px;
    border: 1px solid #ccc;
    overflow-x: auto;
}
</style>

# Grundlegende FastAPI Konzepte: Data Validation, Exceptions

Wir werden weiterhin bei unserer selbstgemachten Bücher API bleiben, jedoch werden wir dieses Projekt nicht so trivial gestallten wie in der Einführung. Wir werden uns mit folgenden Themen beschäftigen:
- Data Validation
- Exception Handling
- Status Codes
- Swagger Configuration
- Python Request Objects

Jetzt werden wir die Bücher als echte Objekte behandeln, dazu werden wir eine eigene "Book" Klasse erstellen. Als Ausgangssituation haben wir den folgenden Code:

```python
from fastapi import FastAPI

app = FastAPI()

BOOKS = []

@app.get("/books")
async def read_all_books():
    return BOOKS
```

Als erstes starten wir den uvicorn Server damit wir unsere API nebenbei testen können:

```
uvicorn books:app --reload
```

<img src="../img/FastAPI_23.png" alt="FastAPI_01" width="500">

Unter der URL sollten wir nun den Endpunkt ansprechen können un eine leere Liste bekommen:

<img src="../img/FastAPI_24.png" alt="FastAPI_01" width="300">

In diesem Beispiel werden wir die Liste "BOOKS" mit Book-Objekten befüllen, dazu erweitern wir unseren Code mit einer "BOOK" Klasse:

```python
from fastapi import FastAPI

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    
    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5),
    Book(4, "HP1", "Author 1", "Book Description", 2),
    Book(5, "HP2", "Author 2", "Book Description", 3),
    Book(6, "HP3", "Author 3", "Book Description", 1)
]

@app.get("/books")
async def read_all_books():
    return BOOKS
```

In der Swagger UI können wir folgendes sehen:

<img src="../img/FastAPI_25.png" alt="FastAPI_01" width="600">

Wir erweitern unsere API noch mit einer POST-Methode, um neue Bücher zu erstellen:

```python
from fastapi import FastAPI, Body

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    
    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5),
    Book(4, "HP1", "Author 1", "Book Description", 2),
    Book(5, "HP2", "Author 2", "Book Description", 3),
    Book(6, "HP3", "Author 3", "Book Description", 1)
]

@app.get("/books")
async def read_all_books():
    return BOOKS

@app.post("/create-book")
async def create_book(book_request=Body()):
    BOOKS.append(book_request)
```

Wir können nun die POST-Methode testen, indem wir folgendes Buch erstellen:

```
  {
    "id": 7,
    "title": "HP4",
    "author": "Author 3",
    "description": "Book Description",
    "rating": 1
  }
```

In der Swager-UI müssen wir bei dem POST-Request den Body mitgeben:

<img src="../img/FastAPI_26.png" alt="FastAPI_01" width="600">

Durch die GET-Methode können wir alle Bücher abrufen und sehen das neue Buch mit der "id=7":

<img src="../img/FastAPI_27.png" alt="FastAPI_01" width="600">

Ein Problem besteht momentan noch, wir haben keine Datenvalidierung in unserer API. Datenvalidierung in FastAPI ist der Prozess, bei dem überprüft wird, ob eingehende Daten (z. B. aus HTTP-Anfragen) den erwarteten Strukturen, Typen und Regeln entsprechen. FastAPI nutzt Pydantic, um dies automatisch und effizient durchzuführen.
<br>
<br>
Datenvalidierung hilft:
- Falsche Eingaben abzufangen (z. B. falsche Datentypen oder fehlende Felder).
- Sicherzustellen, dass alle erforderlichen Daten vorhanden sind.
- Automatisch Fehlermeldungen zu generieren, wenn die Daten nicht validiert werden können.

Wenn wir in unseren momentanen API ein POST-Request erstellen mit dem folgenden Body, wird trotzdem ein Buch angelegt auch wenn die Werte unlogisch erscheinen:

```
  {
    "id": -90,
    "title": "HP4",
    "author": "Author 3",
    "description": "Book Description",
    "rating": 1999
  }
```

<img src="../img/FastAPI_28.png" alt="FastAPI_01" width="600">

Die "id=-90" macht keinen Sinn, hier sollten wohl nur positive ganze Zahlen zugelassen sein. Auch beim "rating=1999" ist wohl ein nicht zulässiger Wert vorhanden, hier sollte ein Intervall festgelegt werden. Aber auch die Datentypen sind zu beachten, wir hätten auch anstelle einer Zahl für eine "id", auch eine Zeichenkette übergeben können.
<br>
<br>
Mit Pydantic erweitern wir unseren Code, mit einer neuen Klasse für die Datenvalidierung:

```python
from fastapi import FastAPI, Body
from pydantic import BaseModel

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    
    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        
class BookRequest(BaseModel):
    id: int
    title: str
    author: str
    description: str
    rating: int
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5),
    Book(4, "HP1", "Author 1", "Book Description", 2),
    Book(5, "HP2", "Author 2", "Book Description", 3),
    Book(6, "HP3", "Author 3", "Book Description", 1)
]

@app.get("/books")
async def read_all_books():
    return BOOKS

@app.post("/create-book")
async def create_book(book_request:BookRequest):
    BOOKS.append(book_request)
```

Die neue Klasse "BookRequest" welche von "BaseModel" erbt, dient als Datenmodell für den POST-Request. "BookRequest" stellt sicher, dass alle Felder vorhanden sind und den erwarteten Datentypen entsprechen. 
```python
class BookRequest(BaseModel):
    id: int
    title: str
    author: str
    description: str
    rating: int
```
Falls ein Nutzer z. B. eine rating-Angabe als string "5" sendet, konvertiert Pydantic sie automatisch in einen int. Falls ein Feld fehlt oder der falsche Typ verwendet wird, gibt FastAPI eine automatische Fehlermeldung zurück.
<br>
<br>
In der Funktion "create_book()" wird nun der Parameter nicht vom Typ "Body()" sein, sondern ein "BookRequest" Objekt. Dadurch wird die Datenvalidierung bei dem Endpunkt "/create-book" durchgeführt, mithilfe der Klasse "BookRequest".
<br>
<br>
Jetzt können wir unseren POST-Request mit der Datenvalidierung in der Swagger-UI testen.
Wir sehen bereits das wir ein Beispielschema für den Body haben:

<img src="../img/FastAPI_29.png" alt="FastAPI_01" width="600">

Wir können jetzt einfach auf "Try it out" und "Execute" klicken, da bereits ein Beispiel für den Bodyinhalt vorhanden ist. Wir bekommen einen HTTP-Statuscode 200 zurück und beim Endpunkt "books" können wir das neue Buch sehen:

<img src="../img/FastAPI_30.png" alt="FastAPI_01" width="600">

Was passiert wenn wir ein Request-Body verwenden, bei dem die Daten von den vorgegebenen Datentypen abweichen? Wir probieren es mit dem Body:

```
{
  "id": "10",
  "title": 123,
  "author": 456",
  "description": "string",
  "rating": 20.33
}
```

Wir bekommen eine Fehlermeldung mit dem HTTP-Statuscode 422:

<img src="../img/FastAPI_31.png" alt="FastAPI_01" width="600">

Betrachten wir einmal den Datentyp von "book_request" näher:

```python
@app.post("/create-book")
async def create_book(book_request:BookRequest):
    print(type(book_request))
    BOOKS.append(book_request)
```

Wir führen den Endpunkt "/create-book" in der Swagger-UI aus und bekommen in der Kosnole ausgegeben:

```
<class 'books.BookRequest'>
```

Wir wissen dass "book_request" vom Typ "BookRequest", also der Klasse welche wir definiert hatten. Wir wollen jedoch dass "book_request" nur vom Typ "Book" ist. Dies erreichen wir durch folgende Änderung:

```python
@app.post("/create-book")
async def create_book(book_request:BookRequest):
    new_book = Book(**book_request.model_dump())
    print(type(new_book))
    BOOKS.append(new_book)
```

Was genau passiert in dem POST-Request?

Der Code-Ausschnitt:
```python
book_request.model_dump()
```
wandelt das "BookRequest" Objekt in ein Dictionary um.

Dann wird durch "**" ein Dictionary-Unpacking durchgeführt:
```python
Book(**book_request.model_dump())
```
dadurch werden die Dictionary-Werte direkt an den "Book" Konstruktor übergeben. Dadurch wird "new_book" zu einer echten Instanz von "Book". In der Konsole bekommen wir dies auch bestätigt:

```
<class 'books.Book'>
```

Das schöne an Pydantic ist, dass wir in der Lage sind, selbst für jedes einzelne Field des Requests, eine Datenvalidierung hinzuzufügen. Wodruch wir dann z.B. eine "id=-50" als nicht zugelassen ansehen können. Dazu müssen wir von Pydantic etwas neues importieren un zwar "Field". Dann können wir im "BookRequest" Modell (Klasse), Field Validierung hinzufügen. Als erstes wollen wir festlegen, dass der "title" immer eine minimale Länge von 3 Zeichen haben muss:

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    
    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        
class BookRequest(BaseModel):
    id: int
    title: str = Field(min_length=3)
    author: str
    description: str
    rating: int
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5),
    Book(4, "HP1", "Author 1", "Book Description", 2),
    Book(5, "HP2", "Author 2", "Book Description", 3),
    Book(6, "HP3", "Author 3", "Book Description", 1)
]

@app.get("/books")
async def read_all_books():
    return BOOKS

@app.post("/create-book")
async def create_book(book_request:BookRequest):
    new_book = Book(**book_request.model_dump())
    BOOKS.append(new_book)
```

Bereits in der Swagger-UI sehen wir im POST-Request die Änderung im Schema:

<img src="../img/FastAPI_32.png" alt="FastAPI_01" width="600">

Wir können noch zu Testzwecken ein POST-Request mit dem folgenden Body durchführen:

```
{
  "id": 0,
  "title": "st",
  "author": "string",
  "description": "string",
  "rating": 0
}
```

Wir bekommen direkt eine dataillierte 422-Fehlermeldung:

<img src="../img/FastAPI_33.png" alt="FastAPI_01" width="600">

Wir fügen noch zu allen anderen Feldern von "BookRequest" eine Daten-Validierung hinzu:

```python
class BookRequest(BaseModel):
    id: int 
    title: str = Field(min_length=3)
    author: str = Field(min_length=1)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=0, lt=6)
```

Wenn wir als Beispiel nochmal ein POST-Request durchführen und den folgenden Body verwenden, bekommen wir ebenfalls eine Fehlermeldung:
```
{
  "id": 110,
  "title": "test",
  "author": "string",
  "description": "string",
  "rating": -3
}
```

<img src="../img/FastAPI_34.png" alt="FastAPI_01" width="600">

In der Fehlermeldung sieht man deutlich dass sich das Problem in dem Field mit dem Namen "rating" handelt:

```
    "type": "greater_than",
    "loc": [
    "body",
    "rating"
    ],
```

Für das "id" Field haben wir bisher doch keine Datenvalidierung vorgenommen. Dafür möchten wir eine Logik implementieren, welche uns eine einzigartige "id" erzeugt, immer wenn wir ein neues Buch erstellen möchten. Dazu erstellen wir uns erstmal eine Funktion:

```python
def find_book_id(book: Book):
    if len(BOOKS) > 0:
        book.id = BOOKS[-1].id + 1
    else:
        book.id = 1
    return book
```

Mithilfe dieser Funktion weisen wir einem neuen Book eine eindeutige "id" zu, basierend auf den bereits existierenden Books in der "BOOKS" Liste. Ketzt müssen wir diese Funktion in der "create_book()" Funkton verwenden:

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    
    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        
class BookRequest(BaseModel):
    id: int 
    title: str = Field(min_length=3)
    author: str = Field(min_length=1)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=0, lt=6)
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5),
    Book(4, "HP1", "Author 1", "Book Description", 2),
    Book(5, "HP2", "Author 2", "Book Description", 3),
    Book(6, "HP3", "Author 3", "Book Description", 1)
]

@app.get("/books")
async def read_all_books():
    return BOOKS

@app.post("/create-book")
async def create_book(book_request:BookRequest):
    new_book = Book(**book_request.model_dump())
    BOOKS.append(find_book_id(new_book))
    
def find_book_id(book: Book):
    if len(BOOKS) > 0:
        book.id = BOOKS[-1].id + 1
    else:
        book.id = 1
    return book
```

Wir starten unsere FastAPI Anwendung neu, damit wir den Ausgangszustand haben mit:

```
uvicorn books:app --reload
```

Jetzt erstellen wir eine POST-Request mit dem folgenden Body:

```
{
  "id": 0,
  "title": "string",
  "author": "string",
  "description": "string",
  "rating": 3
}
```

Wir erhalten aus dem Response einen HTTP-Statuscode von 200, wodurch wir wissen dass unsere Anfrage erfolgreich war. Wenn wir nun mit der GET-Methode alle Bücher auslesen sollte ein neuer Eintrag zu finden sein:

<img src="../img/FastAPI_35.png" alt="FastAPI_01" width="600">

Wir erkennen dass durch die Funktion "find_book_id()" das eben erstellte Buch eine id von "7" hat, anstelle "0". Auch wenn im Request-Body eine id von "0" angegeben wurde.
<br>
<br>
Durch die Verwendung von Ternary-Operators können wir die Funktion "find_book_id()" kürzer gestallten:

```python
def find_book_id(book: Book):
    book.id = 1 if len(BOOKS) == 0 else BOOKS[-1].id + 1
    return book
```

Wir haben auch die Möglichkeit, bestimmte Felder im Request-Body als optional zu kennzeichnen.
Dadurch ist der Benutzer nicht verpflichtet im Request-Body Werte für bestimmte Felder anzugeben.
Dazu müssen wir das Modul "typing" verwenden. Wir wollen dass der Benutzer nicht verpflichtet ist, die "id" im Request-Body anzugeben:

```python
class BookRequest(BaseModel):
    id: Optional[int] = None 
    title: str = Field(min_length=3)
    author: str = Field(min_length=1)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=0, lt=6)
```

Wir testen nun unsere Anwendung indem wir folgenden Request-Body verwenden (ohne "id"):
```
{
  "title": "string",
  "author": "string",
  "description": "string",
  "rating": 3
}
```

<img src="../img/FastAPI_36.png" alt="FastAPI_01" width="600">

Anschließend sehen wir dass neu erstellte Buch, mit einer "id", über die GET-Methode:

<img src="../img/FastAPI_37.png" alt="FastAPI_01" width="600">

Betrachten wir nochmal das Schema des POST-Requests in der Swagger-UI:

<img src="../img/FastAPI_38.png" alt="FastAPI_01" width="600">

Hier sehen wir deutlich das "id" entweder eine ganze Zahl oder Null sein kann. Es wäre schön wenn man dem Benutzer noch mehr Informationen zur "id" bereitstellen könnte. Wir wollen dem Benutzer informieren, dass die "id" nur bei dem POST-Request Optional ist, indem wir eine Beschreibung hinzufügen. Dazu müssen wir in "BookRequest" eine Anpassung durchführen, wir führen ebenfalls bei "id" das "Field" ein:

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import Optional

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    
    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        
class BookRequest(BaseModel):
    id: Optional[int] = Field(description="ID ist not needed on create", default=None)
    title: str = Field(min_length=3)
    author: str = Field(min_length=1)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=0, lt=6)
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5),
    Book(4, "HP1", "Author 1", "Book Description", 2),
    Book(5, "HP2", "Author 2", "Book Description", 3),
    Book(6, "HP3", "Author 3", "Book Description", 1)
]

@app.get("/books")
async def read_all_books():
    return BOOKS

@app.post("/create-book")
async def create_book(book_request:BookRequest):
    new_book = Book(**book_request.model_dump())
    BOOKS.append(find_book_id(new_book))
    
def find_book_id(book: Book):
    book.id = 1 if len(BOOKS) ==  0 else BOOKS[-1].id + 1
    return book
```

In der Swagger-UI sehen wir nun folgendes:

<img src="../img/FastAPI_39.png" alt="FastAPI_01" width="600">

Was noch auffällig ist, sind die vorgegebenen Werte wie z.B. "string":
```
{
  "id": 0,
  "title": "string",
  "author": "string",
  "description": "string",
  "rating": 0
}
```

Wir wollen dem Benutzer jedoch sinnvolle Beispielwerte vorgeben. Dies ist durch "model_config" möglich, es handelt sich um eine spezielle Einstellungskonfiguration für Pydantic-Modelle. Wir füllen also unser "BookRequest" Pydantic-Modell mit zusätzlichen Metadaten für die Swagger-UI Dokumentation:

```python
class BookRequest(BaseModel):
    id: Optional[int] = Field(description="ID ist not needed on create", default=None)
    title: str = Field(min_length=3)
    author: str = Field(min_length=1)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=0, lt=6)
    
    model_config = {
        "json_schema_extra": {
            "example": {
                "title": "A new book",
                "author": "codingwithroby",
                "description": "A new description of a book",
                "rating": 5
            }
        }
    }
```

In der Swagger-UI sehen wir nun diese Metadaten:

<img src="../img/FastAPI_40.png" alt="FastAPI_01" width="600">

Wir erweitern das Projekt weiter, indem wir einen neuen Endpunkt hinzufügen. Wir möchten ein Buch mithilfe der "id" auslesen:

```python
@app.get("/books/{book_id}")
async def read_book(book_id: int):
    for book in BOOKS:
        if book.id == book_id:
            return book
```

Wir testen auch direkt diesen Endpunkt, um sicher zu gehen dass er funktioniert:

<img src="../img/FastAPI_41.png" alt="FastAPI_01" width="600">

Jetzt erweitern wir das Projekt mit einem neuen Endpunkt. Man möchte die Bücher auslesen, durch die Verwendung von "rating":

```python
@app.get("/books/")
async def read_book_by_rating(book_rating: int):
    books_to_return = []
    for book in BOOKS:
        if book.rating == book_rating:
            books_to_return.append(book)
    return books_to_return
```

Auch diesen Endpunkt testen wir in der Swagger-UI:

<img src="../img/FastAPI_42.png" alt="FastAPI_01" width="600">

Außerdem wäre es noch hilfreich, wenn wir unsere Bücher aktualisieren können. Dazu erstellen wir ebenfalls einen neuen Endpunkt:

```python
@app.put("/books/update_book")
async def update_book(book:BookRequest):
    for i in range(len(BOOKS)):
        if BOOKS[i].id == book.id:
            BOOKS[i] = book
```

Um diesen Endpunkt zu testen, werden wir das dritte (id=3) Buch aktualisieren, dazu verwenden wir den folgenden HTTP-Body:
```
{
    "id": 3,
    "title": "How to use a keyboard",
    "author": "codingwithroby",
    "description": "A awesome book!",
    "rating": 5
}
```

<img src="../img/FastAPI_43.png" alt="FastAPI_01" width="600">


Indem wir alle Bücher abrufen, können wir sicher gehen dass unser Buch aktualisiert wurde:

<img src="../img/FastAPI_44.png" alt="FastAPI_01" width="600">

Auch die Funktionalität, ein Buch zu löschen, soll durch einen neuen Endpunkt realisiert werden:

```python
@app.delete("/books/{book_id}")
async def delete_book(book_id: int):
    for i in range(len(BOOKS)):
        if BOOKS[i].id == book_id:
            BOOKS.pop(i)
            break
```

Wir testen den neuen Endpunkt indem wir ein Buch mit der "id=3" löschen:

<img src="../img/FastAPI_45.png" alt="FastAPI_01" width="600">

Anschließend prüfen wir ob dieses Buch auch wirklich nicht mehr vorhanden ist:

<img src="../img/FastAPI_46.png" alt="FastAPI_01" width="600">

Versuchen wir zu Übungszwecken die folgende Aufgabe zu lösen:
- Erstelle ein neues Feld in der Book-Klasse und auch in der BookRequest-Klasse, welches mit "published_date: int" bezeichnet wird. Es handelt sich also um das Veröffentlichungsdatum.
- Jedes Buch muss ein Veröffentlichungsdatum haben!
- Erstelle anschließend einen Endpunkt für eine GET-Request, um die Bücher nach dem Veröffentlichungsdatum zu filtern.

Betrachten wir die Lösung:

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import Optional

app = FastAPI()

class Book():
    id: int
    title: str
    author: str
    description: str
    rating: int
    published_date: int
    
    def __init__(self, id, title, author, description, rating, published_date):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating
        self.published_date = published_date
        
class BookRequest(BaseModel):
    id: Optional[int] = Field(description="ID ist not needed on create", default=None)
    title: str = Field(min_length=3)
    author: str = Field(min_length=1)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=0, lt=6)
    published_date: int = Field(gt=1999, lt=2031)
    
    model_config = {
        "json_schema_extra": {
            "example": {
                "title": "A new book",
                "author": "codingwithroby",
                "description": "A new description of a book",
                "rating": 5,
                "published_date": 2029
            }
        }
    }
        
BOOKS = [
    Book(1, "Computer Science Pro", "codingwithroby", "A very nice book!", 5, 2030),
    Book(2, "Be Fast with FastAPI", "codingwithroby", "A great book!", 5, 230),
    Book(3, "Master Endpoints", "codingwithroby", "A awesome book!", 5, 2029),
    Book(4, "HP1", "Author 1", "Book Description", 2, 2028),
    Book(5, "HP2", "Author 2", "Book Description", 3, 2027),
    Book(6, "HP3", "Author 3", "Book Description", 1, 2026)
]

@app.get("/books")
async def read_all_books():
    return BOOKS

@app.get("/books/{book_id}")
async def read_book(book_id: int):
    for book in BOOKS:
        if book.id == book_id:
            return book
        
@app.get("/books/publish/")
async def read_books_by_publish_date(published_date: int):
    books_to_return = []
    for book in BOOKS:
        if book.published_date == published_date:
            books_to_return.append(book)
    return books_to_return

@app.post("/create-book")
async def create_book(book_request:BookRequest):
    new_book = Book(**book_request.model_dump())
    BOOKS.append(find_book_id(new_book))
    
def find_book_id(book: Book):
    book.id = 1 if len(BOOKS) ==  0 else BOOKS[-1].id + 1
    return book

@app.get("/books/")
async def read_book_by_rating(book_rating: int):
    books_to_return = []
    for book in BOOKS:
        if book.rating == book_rating:
            books_to_return.append(book)
    return books_to_return

@app.put("/books/update_book")
async def update_book(book:BookRequest):
    for i in range(len(BOOKS)):
        if BOOKS[i].id == book.id:
            BOOKS[i] = book
            
@app.delete("/books/{book_id}")
async def delete_book(book_id: int):
    for i in range(len(BOOKS)):
        if BOOKS[i].id == book_id:
            BOOKS.pop(i)
            break
        
```

Wir Testen unseren neuen Endpunkt, indem wir uns alle Bücher mit dem Datum "2030" ausgeben lassen:

<img src="../img/FastAPI_47.png" alt="FastAPI_01" width="600">

Natürlich sollte man an dieser Stelle noch prüfen, ob alle anderen Endpunkte noch funktionieren! Wir haben immerhin etwas an den grundlegenden Klassen geändert. 